## Daily Impacts

In [71]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import time
import timeit
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
import re

In [72]:
results_scene_1 = pd.read_csv('Final_results/results_scenario1_0.05_1.5.csv',index_col=0)#,usecols=range(6))
results_scene_1.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
3-4,0.0,0.0,0.0,3.877147,0.249570,1.873283
3-7,0.0,0.0,0.0,12.012387,1.458571,22.529042
3-9,0.0,0.0,0.0,0.140546,0.072559,1.786895
3-10,0.0,0.0,0.0,0.157064,0.124834,2.718103
3-11,0.0,0.0,0.0,3.853464,0.145975,0.000561


In [73]:
results_scene_2 = pd.read_csv('Final_results/results_scenario2_0.05_1.5.csv',index_col=0,usecols=range(1,8))
results_scene_2.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
3-4,0.0,0.0,0.0,3.877147,0.249570,1.873283
3-13,0.0,0.0,0.0,11.617495,0.256696,1.125809
3-43,0.0,0.0,0.0,6.756582,0.364152,1.879266
3-45,0.0,0.0,0.0,9.805379,0.187292,0.007330
3-48,0.0,0.0,0.0,49.069609,2.608330,13.322061


In [74]:
results_scene_3 = pd.read_csv('Final_results/results_scenario3_with_scenario1_0.05_1.5.csv',index_col=0)#,usecols=range(6))
results_scene_3.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
3-4,0.0,0.0,0.0,4.492907,0.249570,1.257523
3-7,0.0,0.0,0.0,12.012387,1.458571,22.529042
3-9,0.0,0.0,0.0,0.140546,0.072559,1.786895
3-10,0.0,0.0,0.0,0.157064,0.124834,2.718103
3-11,0.0,0.0,0.0,3.853464,0.145975,0.000561


In [75]:
wages = pd.read_csv('../modeling/final_allMode_with_2017wage_cleaned_update.csv')
wages.head(5)

,Unnamed: 0,DOlocationID,ODpair,PUlocationID,duration,mode,nest,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000
0,0,4,3-4,3,39.695000,2,1,64.000,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
1,1,4,3-4,3,45.216667,3,1,61.500,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
2,2,4,3-4,3,83.000000,4,2,5.500,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
3,3,4,3-4,3,225.933333,5,3,0.000,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
4,4,4,3-4,3,39.695000,6,4,29.424,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005


In [76]:
# agg_scene_1 = results_scene_1.drop('destination',axis=1).groupby('origin').sum()
# agg_scene_1.head()
# agg_scene_1.columns = [name + '_predicted' for name in agg_scene_1.columns]
# acs = pd.read_csv('final_acs_transportation_choice.csv',index_col=0)
# acs.head()
# acs.columns = [name + '_actual' for name in acs.columns]
# scene_1_combined = agg_scene_1.merge(acs,how='left',left_index=True,right_index=True)
# scene_1_combined.head()

In [77]:
# Renaming for convenience in calculation
results_scene_1.columns = [re.sub(r'[^\d.]+','',col) for col in results_scene_1.columns]
results_scene_2.columns = [re.sub(r'[^\d.]+','',col) for col in results_scene_2.columns]
results_scene_3.columns = [re.sub(r'[^\d.]+','',col) for col in results_scene_3.columns]

In [78]:
results_scene_1.reset_index(inplace=True)
results_scene_2.reset_index(inplace=True)
results_scene_3.reset_index(inplace=True)
results_scene_1.rename_axis({'index':'ODpair'},axis=1,inplace=True)
results_scene_2.rename_axis({'index':'ODpair'},axis=1,inplace=True)
results_scene_3.rename_axis({'index':'ODpair'},axis=1,inplace=True)

In [79]:
results_scene_1.head()

,ODpair,1,2,3,4,5,6
0,3-4,0.0,0.0,0.0,3.877147,0.249570,1.873283
1,3-7,0.0,0.0,0.0,12.012387,1.458571,22.529042
2,3-9,0.0,0.0,0.0,0.140546,0.072559,1.786895
3,3-10,0.0,0.0,0.0,0.157064,0.124834,2.718103
4,3-11,0.0,0.0,0.0,3.853464,0.145975,0.000561


In [80]:
# Getting the origin taxi zone
results_scene_1['origin'] = results_scene_1.ODpair.apply(lambda x: x.split('-')[0]).astype('int')
results_scene_2['origin'] = results_scene_2.ODpair.apply(lambda x: x.split('-')[0]).astype('int')
results_scene_3['origin'] = results_scene_3.ODpair.apply(lambda x: x.split('-')[0]).astype('int')

In [81]:
results_scene_1.head()

,ODpair,1,2,3,4,5,6,origin
0,3-4,0.0,0.0,0.0,3.877147,0.249570,1.873283,3
1,3-7,0.0,0.0,0.0,12.012387,1.458571,22.529042,3
2,3-9,0.0,0.0,0.0,0.140546,0.072559,1.786895,3
3,3-10,0.0,0.0,0.0,0.157064,0.124834,2.718103,3
4,3-11,0.0,0.0,0.0,3.853464,0.145975,0.000561,3


In [82]:
# Aggregating the number of people in each ODpair by the origin zone
agg_scene_1 = results_scene_1.groupby('origin').sum()
agg_scene_2 = results_scene_2.groupby('origin').sum()
agg_scene_3 = results_scene_3.groupby('origin').sum()

In [83]:
agg_scene_1.columns = agg_scene_1.columns.astype('int')
agg_scene_2.columns = agg_scene_2.columns.astype('int')
agg_scene_3.columns = agg_scene_3.columns.astype('int')

In [84]:
agg_scene_1.head()

,1,2,3,4,5,6
origin,,,,,,
3,17.700048,94.582320,0.017569,4008.909286,753.130953,3831.659823
4,1514.125930,17.319027,0.001373,3592.415229,2614.098018,2879.040422
5,0.000000,41.423708,1.905258,2794.741855,389.978755,4999.950424
6,229.753956,0.005788,24.312307,2275.062656,171.020749,2568.844544
7,697.991501,7.201573,3.055054,21513.776959,1360.529266,13105.445646


In [85]:
pickup_list = wages.PUlocationID.unique()

In [86]:
origin_time_and_money_avg = wages.groupby(['PUlocationID','mode'],as_index=False)['duration','price'].mean()

In [87]:
origin_time_and_money_avg.head()

,PUlocationID,mode,duration,price
0,3,1,38.302359,37.428045
1,3,2,37.718869,56.582192
2,3,3,41.066196,50.717949
3,3,4,81.756356,5.948623
4,3,5,268.553355,0.000000


In [88]:
agg_scene_1.shape

(256, 6)

In [89]:
len(pickup_list)

256

## Status quo Commute Statistics

In [90]:
pivoted_time = origin_time_and_money_avg.reset_index().pivot(index='PUlocationID',columns='mode',values='duration')

In [91]:
commute_time = pivoted_time.multiply(agg_scene_1).sum().sum()

In [92]:
avg_commute_time = commute_time/agg_scene_1.sum().sum()
avg_commute_time

71.33244721382114

In [93]:
pivoted_money = origin_time_and_money_avg.reset_index().pivot(index='PUlocationID',columns='mode',values='price')

In [94]:
commute_cost = pivoted_money.multiply(agg_scene_1).sum().sum()

In [95]:
avg_commute_cost = commute_cost/agg_scene_1.sum().sum()
avg_commute_cost

10.171392416247677

## Subway ridership

In [50]:
agg_scene_1.sum()[4]

1448265.8859749658

In [51]:
agg_scene_2.sum()[4]

1466962.6905047998

In [52]:
agg_scene_3.sum()[4]

1568531.5772572262

In [96]:
agg_scene_2.sum()[4] - agg_scene_1.sum()[4]

18696.804529834073

In [97]:
agg_scene_3.sum()[4] - agg_scene_1.sum()[4]

120265.69128226046

In [98]:
agg_scene_3.sum()[4] - agg_scene_2.sum()[4]

101568.88675242639

In [99]:
(agg_scene_2.sum()[4] - agg_scene_1.sum()[4])/agg_scene_1.sum()[4] * 100

1.2909787291749588

In [100]:
(agg_scene_3.sum()[4] - agg_scene_1.sum()[4])/agg_scene_1.sum()[4] * 100

8.304116837033565

In [101]:
(agg_scene_3.sum()[4] - agg_scene_2.sum()[4])/agg_scene_2.sum()[4] * 100

6.9237539175229665

### Scenario 1 VS 2

In [105]:
time_diff = 0
money_diff = 0
for zone in tqdm(pickup_list,position=0):
    for col in agg_scene_1.columns:
        diff = (agg_scene_2.loc[zone,col]-agg_scene_1.loc[zone,col])
        time_and_money = (origin_time_and_money_avg.loc[(origin_time_and_money_avg['PUlocationID']==zone)\
                                                       & (origin_time_and_money_avg['mode']==col)])
        time_diff+= diff * (0 if time_and_money.empty else time_and_money['duration'].values[0])
        money_diff+= diff * (0 if time_and_money.empty else time_and_money['price'].values[0])

100%|██████████| 256/256 [00:01<00:00, 193.34it/s]


In [106]:
time_diff

664182.824898697

In [107]:
money_diff

-572010.3659846579

In [108]:
population = agg_scene_1.sum().sum()

In [109]:
# Extra commute time per person
time_diff/population

0.23051090439821367

In [110]:
time_diff/population/avg_commute_time *100

0.3231501419084226

In [111]:
# Money saved per person
money_diff/population

-0.19852158448750543

In [112]:
money_diff/population/avg_commute_cost *100

-1.9517640885665672

### Scenario 1 VS 3

In [113]:
time_diff = 0
money_diff = 0
for zone in tqdm(pickup_list,position=0):
    for col in agg_scene_1.columns:
        diff = (agg_scene_3.loc[zone,col]-agg_scene_1.loc[zone,col])
        time_and_money = (origin_time_and_money_avg.loc[(origin_time_and_money_avg['PUlocationID']==zone)\
                                                       & (origin_time_and_money_avg['mode']==int(col))])
        time_diff+= diff * (0 if time_and_money.empty else time_and_money['duration'].values[0])
        money_diff+= diff * (0 if time_and_money.empty else time_and_money['price'].values[0])

100%|██████████| 256/256 [00:01<00:00, 202.64it/s]


In [114]:
time_diff

4145773.768297022

In [115]:
money_diff

-1704369.9815825045

In [116]:
population = agg_scene_2.sum().sum()

In [117]:
# Extra commute time per person
time_diff/population

1.4388298295823805

In [118]:
time_diff/population/avg_commute_time *100

2.0170762195630907

In [119]:
# Money saved per person
money_diff/population

-0.5915176532059108

In [120]:
money_diff/population/avg_commute_cost *100

-5.815503217249062

### Scenario 2 VS 3

In [121]:
time_diff = 0
money_diff = 0
for zone in tqdm(pickup_list,position=0):
    for col in agg_scene_2.columns:
        diff = (agg_scene_3.loc[zone,col]-agg_scene_2.loc[zone,col])
        time_and_money = (origin_time_and_money_avg.loc[(origin_time_and_money_avg['PUlocationID']==zone)\
                                                       & (origin_time_and_money_avg['mode']==int(col))])
        time_diff+= diff * (0 if time_and_money.empty else time_and_money['duration'].values[0])
        money_diff+= diff * (0 if time_and_money.empty else time_and_money['price'].values[0])

100%|██████████| 256/256 [00:01<00:00, 202.61it/s]


In [122]:
time_diff

3481590.9433983313

In [123]:
money_diff

-1132359.615597848

In [124]:
population = agg_scene_2.sum().sum()

In [125]:
# Extra commute time per person
time_diff/population

1.208318925184169

In [126]:
time_diff/population/(avg_commute_time+0.23051090439821367) *100

1.6884697851478847

In [127]:
# Money saved per person
money_diff/population

-0.39299606871840603

In [128]:
money_diff/population/(avg_commute_cost-0.19852158448750543) *100

-3.9406513465194832

#### Use below code only if you want exact values for the difference in time/money for commuters as it takes a very long time to run (>3 hours). It can also be used to get the difference for each taxi zone (after some minor changes).

In [148]:
# time_total = 0
# money_total = 0
# for pair in tqdm(ODpair_list,position=0):
#     time_pair = 0
#     money_pair = 0
#     for col in results_scene_1.columns:
#         time_pair+=(results_scene_1.loc[results_scene_1.index==pair,col]-results_scene_2.loc[results_scene_2.index==pair,col])\
#         *wages.loc[(wages['ODpair']==pair) & wages['mode']==int(col),'duration']
#         money_pair+=(results_scene_1.loc[results_scene_1.index==pair,col]-results_scene_2.loc[results_scene_2.index==pair,col])\
#         *wages.loc[(wages['ODpair']==pair) & wages['mode']==int(col),'price']
#     time_total+=time_pair
#     money_total+=money_pair